In [5]:
import sys
sys.path.insert(0, "..")
from utils import linalg
from importlib import reload
import distributed
from distributed import kernel
reload(distributed.sharded_matrix)
from distributed.sharded_matrix import ShardedMatrix, ShardedSymmetricMatrix
from distributed import matmul
import numpy as np
import pywren
from utils import misc
import math
import concurrent.futures as fs
import time
import os
from numba import jit
import scipy.linalg
from sklearn import metrics
from scipy.linalg import LinAlgError

from distributed import bcd
reload(bcd)

<module 'distributed.bcd' from '../distributed/bcd.py'>

In [6]:
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')

In [2]:
%time X_block_sharded = ShardedMatrix("X_block_scrambled_3_3_pool_12_12_patch_stride_1_1_block_no_normalize",\
                                shard_size_0=4096,\
                                bucket="imagenet-raw",\
                                n_jobs=32)

CPU times: user 100 ms, sys: 12 ms, total: 112 ms
Wall time: 184 ms


In [13]:
x = X_block_sharded.get_block(0,0)

In [6]:
x  = np.memmap("/dev/shm/block0", dtype="float64", mode="r")

In [11]:
x = x.reshape(-1, 4096*4)

In [20]:
idxs = np.random.choice(4096, 4096, replace=False)
one = idxs[:2048]
two = idxs[2048:]


0.01 * np.median(np.linalg.norm(x[one] - x[two], axis=1))

1.1107832336425782

In [55]:
GAMMA = 1e0
NUM_TRAIN = 1281167

In [ ]:
# This if branch should take 7 minutes
%time X_block_0 = np.load("/home/ubuntu/imagenet-data/X_train_lift_3_3_pool_12_12_patch_stride_1.npy")
%time idxs = np.load("/home/ubuntu/imagenet-data/scramble_idxs.npy")

In [ ]:
%time X_train = X_block_0.reshape(NUM_TRAIN, -1)[idxs]
%time X_train = X_train.astype('float64')
%time X_train_norms = np.power(np.linalg.norm(X_train, axis=1), 2)

In [ ]:
%time idxs = np.load("/home/ubuntu/imagenet-data/scramble_idxs.npy")
%time labels_train = np.load("/home/ubuntu/imagenet-data/imagenet_train_labels.npy")[idxs]
%time y_train = np.eye(1000)[labels_train.astype('int')]

In [ ]:
del X_block_0

In [ ]:
%time X_test = np.load("/home/ubuntu/imagenet-data/X_test_lift_3_3_pool_12_12_patch_stride_1.npy")
%time X_test = X_test.astype('float64')
%time X_test_norms = np.linalg.norm(X_test, axis=1)
%time labels_test = np.load("/home/ubuntu/imagenet-data/imagenet_test_labels.npy")

In [ ]:
X_test /= np.sqrt(X_test.shape[1])
X_train /= np.sqrt(X_test.shape[1])

In [ ]:
%time X_test_norms = np.power(np.linalg.norm(X_test, axis=1), 2)
%time X_train_norms = np.power(np.linalg.norm(X_train, axis=1), 2)

In [ ]:
X_test.shape

In [ ]:
K_test = linalg.computeRBFGramMatrix(XTest=X_test, XTrain=X_train, gamma=1e0)

In [19]:
np.argmax(y_pred_test, axis=1)]

0

In [12]:
model = np.load("/home/ubuntu/model_3_3_pool_12_12_patch_stride_1_gamma_1e0_lambda_1e-2_train_acc_0.249_next_block_acc_0.175.npy")

In [13]:
y_pred_test = K_test.dot(model)

In [24]:
metrics.accuracy_score(labels_test[:100], np.argmax(y_pred_test[:100], axis=1))

0.11

In [23]:
np.argmax(y_pred_test[:100], axis=1)

array([  0,   0,   0,   0, 640, 111, 640, 278, 768, 640, 111, 105, 111,
       640, 992, 995, 768,   0, 376,   0,   0, 105, 339, 873, 944,   0,
         0, 685, 111, 278, 105, 105, 389, 105, 640, 640, 116, 111, 111,
       111,   0, 179, 111, 640,   0, 278, 316, 179, 111, 389, 963,  83,
       640, 640,  95, 959, 963, 109, 640, 352, 640, 352, 640, 987, 312,
       339, 640, 278, 317, 640, 685, 495, 685, 121, 406,  83, 640,   6,
        95, 142,  83, 116, 959,  55, 930, 986, 640, 959, 640, 963,  83,
       640, 386, 138, 431,  83, 151,   0, 111, 138])

In [45]:
np.random.seed(0)
small_test_idxs = np.random.choice(K_test.shape[0], 5000, replace=False)
tiny_test_idxs = np.random.choice(K_test.shape[0], 500, replace=False)

In [26]:
K_test_tiny = K_test[tiny_test_idxs, :]

In [35]:
K_test_small

array([[  5.19381529e-03,   2.41049127e-03,   1.00381841e-02, ...,
          5.39413905e-03,   3.12326840e-02,   1.40564271e-03],
       [  3.51658295e-04,   2.30393321e-04,   2.06582845e-03, ...,
          8.77997388e-04,   1.59257323e-03,   5.35640941e-05],
       [  2.72380756e-02,   2.94913158e-03,   9.83917214e-02, ...,
          1.17548433e-02,   1.31568407e-02,   1.17339451e-03],
       ..., 
       [  5.35817409e-02,   1.38668514e-01,   4.72292236e-02, ...,
          2.66572831e-01,   6.62012709e-02,   3.39091975e-02],
       [  1.01597791e-02,   1.06262018e-01,   2.10669543e-02, ...,
          1.46890848e-01,   3.37509807e-03,   1.46583554e-03],
       [  1.11348082e-01,   8.60037181e-03,   1.51008573e-01, ...,
          1.96270832e-02,   8.88218639e-02,   1.88914472e-02]])

In [46]:
X_train_tiny = X_train[tiny_test_idxs,:]

In [82]:
reload(linalg)

<module 'utils.linalg' from '../utils/linalg.py'>

In [48]:
%time K_train_tiny = linalg.computeRBFGramMatrix(XTest=X_train_tiny, XTrain=X_train, gamma=1e0, sq_norms_train=X_train_norms)

../utils/linalg.py:20: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if sq_norms_train == None:


CPU times: user 35min 40s, sys: 21 s, total: 36min 1s
Wall time: 1min 2s


In [50]:
%time y_train_tiny_pred = K_train_tiny.dot(model)

CPU times: user 2min 10s, sys: 1.65 s, total: 2min 11s
Wall time: 2.58 s


In [51]:
metrics.accuracy_score(labels_train[tiny_test_idxs], np.argmax(y_train_tiny_pred, axis=1))

0.089999999999999997

In [53]:
from distributed import bcd

In [7]:
K_sharded = distributed.sharded_matrix.ShardedSymmetricMatrix("rbf(XXT(X_block_scrambled_3_3_pool_12_12_patch_stride_1_1_blocks_no_normalize_04.22.2017, 1.0)",
                                                              bucket="vaishaalpywrenlinalg"
                                                              )

In [38]:

futures = bcd.fast_kernel_column_block_async(K_sharded, col_blocks=[300])

In [39]:
fs.wait(futures)

DoneAndNotDoneFutures(done={<Future at 0x7fb61249e860 state=finished returned tuple>, <Future at 0x7fb6124a4080 state=finished returned tuple>, <Future at 0x7fb61249e908 state=finished returned tuple>, <Future at 0x7fb6124a4128 state=finished returned tuple>, <Future at 0x7fb61249e9b0 state=finished returned tuple>, <Future at 0x7fb6124a41d0 state=finished returned tuple>, <Future at 0x7fb61249ea58 state=finished returned tuple>, <Future at 0x7fb6124a4278 state=finished returned tuple>, <Future at 0x7fb61249eb00 state=finished returned tuple>, <Future at 0x7fb6124a4320 state=finished returned tuple>, <Future at 0x7fb61249eba8 state=finished returned tuple>, <Future at 0x7fb6124a43c8 state=finished returned tuple>, <Future at 0x7fb61249ec50 state=finished returned tuple>, <Future at 0x7fb6124a4470 state=finished returned tuple>, <Future at 0x7fb61249ecf8 state=finished returned tuple>, <Future at 0x7fb6124a4518 state=finished returned tuple>, <Future at 0x7fb61249eda0 state=finished ret

In [40]:
K_block = bcd.load_mmap(*futures[0].result())

In [41]:
K_block.shape

(1281167, 4096)

In [21]:
model = np.load("/home/ubuntu/model_3_3_pool_12_12_patch_stride_1_gamma_1e0_lambda_1e-2_train_acc_0.249_next_block_acc_0.175.npy")

In [26]:
%time y_block_pred = K_block.T.dot(model)

CPU times: user 9min 42s, sys: 45min 10s, total: 54min 53s
Wall time: 53.5 s


In [31]:
y_train_block = labels_train[275*4096:276*4096]

In [33]:
np.argmax(y_block_pred, axis=1)

array([111, 111, 995, ..., 111, 111, 768])

In [36]:
metrics.accuracy_score(np.argmax(y_block_pred, axis=1), y_train_block)

0.00341796875

In [46]:
metrics.accuracy_score(np.argmax(model[275*4096:276*4096, :], axis=1), y_train_block)

0.999755859375